<a href="https://colab.research.google.com/github/VHDRKL/1031/blob/main/1091115_%E8%82%A1%E7%A5%A8%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() #0050.TW.csv

Saving 0050.TW.csv to 0050.TW.csv


{'0050.TW.csv': b'Date,Open,High,Low,Close,Adj Close,Volume\n2019-11-13,91.550003,91.650002,91.050003,91.449997,88.572029,1935545\n2019-11-14,91.599998,91.800003,91.199997,91.400002,88.523605,2174281\n2019-11-15,92.000000,92.500000,92.000000,92.300003,89.395287,2510879\n2019-11-18,92.500000,92.900002,92.400002,92.900002,89.976402,2087613\n2019-11-19,92.900002,93.599998,92.650002,93.599998,90.654366,2821624\n2019-11-20,93.599998,93.599998,93.099998,93.250000,90.315384,7954619\n2019-11-21,92.300003,92.550003,91.800003,92.500000,89.588989,3283838\n2019-11-22,92.550003,92.750000,92.400002,92.550003,89.637413,1506544\n2019-11-25,93.000000,93.099998,92.550003,92.599998,89.685837,2520694\n2019-11-26,92.949997,93.300003,92.800003,92.849998,89.927971,3827480\n2019-11-27,92.849998,93.199997,92.750000,93.199997,90.266953,1957676\n2019-11-28,93.250000,93.250000,92.800003,92.949997,90.024818,1352720\n2019-11-29,92.849998,92.849998,91.449997,91.500000,88.620461,5344208\n2019-12-02,91.949997,92.15000

In [ ]:
import pandas as pd
import numpy as np
stock=pd.read_csv('0050.TW.csv')


In [ ]:
stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-11-13,91.550003,91.650002,91.050003,91.449997,88.572029,1935545.0
1,2019-11-14,91.599998,91.800003,91.199997,91.400002,88.523605,2174281.0
2,2019-11-15,92.000000,92.500000,92.000000,92.300003,89.395287,2510879.0
3,2019-11-18,92.500000,92.900002,92.400002,92.900002,89.976402,2087613.0
4,2019-11-19,92.900002,93.599998,92.650002,93.599998,90.654366,2821624.0


In [ ]:
import numpy as np
def normalize(df):
  norm=df.apply( lambda x: (x-np.min(x)) / (np.max(x)-np.min(x)) )
  return norm

In [ ]:
stock = stock.dropna()
min1=np.min(stock['Close'])
scale=np.max(stock['Close'])- min1
stock.iloc[:,1:7] = normalize(stock.iloc[:,1:7])
stock['y']=stock['Close'].shift(-1)
stock.head()

In [ ]:
stock=stock.drop(labels='Date',axis=1)
stock=stock.drop(labels='Open',axis=1)
stock=stock.drop(labels='Adj Close',axis=1)
stock.head()


In [ ]:
train=stock[0:200]
len(train) #200筆
test=stock[-47:]
len(test) #47筆

47

In [ ]:
import tensorflow as tf
data1=tf.keras.preprocessing.sequence.TimeseriesGenerator(train, 
 train,length=7,sampling_rate=1,stride=1  )
test1=tf.keras.preprocessing.sequence.TimeseriesGenerator(test, 
 test,length=7,sampling_rate=1, stride=1   )

In [ ]:
def train_window(df, days=7, pday=1):
  x_train, y_train=[],[]
  for i in range(df.shape[0]-days - pday +1 ):
    x_train.append( np.array(df.iloc[i:i+days,0:-1]  )  )
    y_train.append( np.array(df.iloc[i:i+days,-1] )  ) 
  return np.array(x_train ), np.array(y_train)

In [ ]:
(x_train, y_train)=train_window(train)
(x_test, y_test)=train_window(test)
stock.head()

,High,Low,Close,Volume,y
0,0.537760,0.574186,0.562653,0.027441,0.561425
1,0.541667,0.577805,0.561425,0.030953,0.583538
2,0.559896,0.597105,0.583538,0.035906,0.598280
3,0.570313,0.606755,0.598280,0.029678,0.615479
4,0.588542,0.612787,0.615479,0.040478,0.606880


In [ ]:
x_train.shape, y_train.shape #((193, 7, 4), (193, 7))
x_test.shape, y_test.shape  #((40, 7, 4), (40, 7))

((40, 7, 4), (40, 7))

In [ ]:
class MyRNN(tf.keras.Model):
  def __init__(self, units):
    super().__init__()
    self.f1=tf.keras.layers.SimpleRNN(units, input_shape=(7,4), dropout=0.2,
         return_sequences=True, return_state=True,unroll=True)
    self.f2=tf.keras.layers.SimpleRNN(units, dropout=0.2,unroll=True)
    self.f3=tf.keras.layers.Dense(32, 'relu')
    self.f4=tf.keras.layers.Dense(1, 'sigmoid')

  def call(self, inputs):
    x=self.f1(inputs)
    x=self.f2(x)
    x=self.f3(x)
    x=self.f4(x)
    return x 

In [ ]:
batchsz=16
data=tf.data.Dataset.from_tensor_slices( (x_train, y_train) )
data = data.batch(batchsz,drop_remainder=True)
data_test=tf.data.Dataset.from_tensor_slices( (x_test, y_test) )
data_test = data_test.batch(batchsz,drop_remainder=True)
data_iter = iter(data)
samples = next(data_iter)
#print(samples[0].shape,samples[1].shape)
model=MyRNN(32)
model.compile(optimizer='adam', loss='mae', metrics=['mse'])
model.fit(data, epochs=100, validation_data=data_test, shuffle=True)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

12/12 [==============================] - 0s 23ms/step - loss: 0.1846 - mse: 0.0491 - val_loss: 0.2672 - val_mse: 0.0730
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 0.1410 - mse: 0.0307 - val_loss: 0.3261 - val_mse: 0.1077
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 0.1279 - mse: 0.0246 - val_loss: 0.2403 - val_mse: 0.0590
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 0.1186 - mse: 0.0237 - val_loss: 0.2298 -

<pre>
Date	Open	High	Low	Close	Adj Close	Volume	y
0	2019-11-13	91.550003	91.650002	91.050003	91.449997	88.572029	1935545.0	91.400002
1	2019-11-14	91.599998	91.800003	91.199997	91.400002	88.523605	2174281.0	92.300003
2	2019-11-15	92.000000	92.500000	92.000000	92.300003	89.395287	2510879.0	92.900002
3	2019-11-18	92.500000	92.900002	92.400002	92.900002	89.976402	2087613.0	93.599998
4	2019-11-19	92.900002	93.599998	92.650002	93.599998	90.654366	2821624.0	93.250000
</pre>